## PAGA Basic Analysis (Only Main Analysis)

- In this notebook, I include basic leiden clustering with no downstream timeseries analysis

- Mean parameter values are projected onto the space to visualize

- Distributions of genes of known function should be assembled in this notebook

- Finally, basic clustering and ontology enrichment are present at the end

- This notebook should be used to decide on clustering parameters and the resulting paga_df saved to disk

In [ ]:
import paulssonlab.deaton.trenchripper.trenchripper as tr

import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import sklearn as skl
import dask.dataframe as dd
import dask.array as da
import dask
import warnings
import copy
import random
from sklearn.metrics.pairwise import (
    euclidean_distances,
    manhattan_distances,
    cosine_distances,
)

from sklearn.metrics import silhouette_score
import scipy.stats
from sklearn.linear_model import LinearRegression
from sklearn.cluster import AffinityPropagation
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering

import scanpy as sc
import anndata
import scipy as sp
import scipy.sparse
import dask.array as da
from igraph.drawing.text import TextDrawer
from tslearn.neighbors import KNeighborsTimeSeries
from tslearn.metrics import cdist_soft_dtw_normalized, cdist_soft_dtw
import networkx as nx
import igraph as ig
import leidenalg
import umap
from scanpy.plotting.palettes import default_20, vega_20_scanpy
from matplotlib import pyplot as plt
import ast
from tslearn.barycenters import (
    softdtw_barycenter,
    dtw_barycenter_averaging,
    euclidean_barycenter,
)


import pylab
import scipy.cluster.hierarchy as sch

import matplotlib.gridspec as gridspec
import matplotlib as mpl

import holoviews as hv

hv.extension("bokeh")
random.seed(42)
np.random.seed(42)

warnings.filterwarnings(action="once", category=UserWarning)

### Load Data From First Notebook

In [ ]:
paga_df_only = pd.read_pickle("./2021-12-07_paga_df_only.pkl")
paga_df = sc.read("./2021-12-07_paga_df.h5ad")
paga_df.obs = copy.deepcopy(paga_df_only)

In [ ]:
paga_df_only["Target Sites"] = paga_df_only["Target Sites"].apply(
    lambda x: ast.literal_eval(x)
)

In [ ]:
fig = sc.pl.umap(
    paga_df,
    color=["leiden_lowres", "leiden", "leiden_highres", "leiden_ultrahighres"],
    title=[
        "Leiden Resolution=0.25",
        "Leiden Resolution=1.",
        "Leiden Resolution=1.5",
        "Leiden Resolution=3.",
    ],
    show=False,
    legend_loc="on data",
    edges=True,
    add_outline=False,
    size=50,
    return_fig=True,
    palette=vega_20_scanpy,
)

### Gene Browser

In [ ]:
gene_browser_df = copy.deepcopy(paga_df.obs)
gene_browser_df = gene_browser_df.reset_index()

In [ ]:
gene_browser_df = gene_browser_df[
    [
        "sgRNAid",
        "TargetID",
        "Gene",
        "phenotype trenchids",
        "N Mismatch",
        "N Target Sites",
        "Category",
        "Strand",
        "leiden_lowres",
        "leiden",
        "leiden_highres",
        "leiden_ultrahighres",
    ]
]
gene_browser_df["sgRNAid"] = gene_browser_df["sgRNAid"].astype(int)

In [ ]:
kymo_xarr = tr.kymo_xarr(
    "/home/de64/scratch/de64/sync_folder/2021-11-08_lDE20_Final_3/Growth_Division/"
)
wrapped_kymo_xarr = tr.kymo_xarr(
    "/home/de64/scratch/de64/sync_folder/2021-11-08_lDE20_Final_3/Growth_Division/",
    unwrap=False,
)

In [ ]:
(
    gene_table_layout,
    select_gene,
    select_trenchid,
    select_unpacked_trenchid,
) = tr.linked_table(
    gene_browser_df,
    index_key="leiden_ultrahighres",
    trenchids_as_list=True,
    trenchid_column="phenotype trenchids",
)
# gene_table_layout,select_gene,select_trenchid,select_unpacked_trenchid = tr.linked_table(gene_browser_df,index_key='Gene',trenchids_as_list=True,trenchid_column='phenotype trenchids')

In [ ]:
gene_table_layout

In [ ]:
output_display, save_button = tr.linked_kymograph_for_table(
    kymo_xarr,
    wrapped_kymo_xarr,
    gene_browser_df,
    select_gene,
    select_trenchid,
    index_key="leiden_ultrahighres",
    select_unpacked_trenchid=select_unpacked_trenchid,
    trenchid_column="phenotype trenchids",
    y_scale=3,
    x_window_size=300,
)

In [ ]:
output_display

In [ ]:
save_button